In [10]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

In [11]:
dataframes = pd.read_parquet('D:/escritorio/STEAM/parquet/engineer.parquet')
dataframes.head(3)

,item_id,playtime_forever,items_count,genres,app_name,price,developer,release_year,recommend,sentiment_analysis
0,70,0,109,Action,Half-Life,9.99,Valve,1998,True,2
1,70,0,109,Action,Half-Life,9.99,Valve,1998,True,0
7,70,0,109,Action,Half-Life,9.99,Valve,1998,True,1


In [12]:
#Creo un nuevo dataframe con las columnas necesarias para el modelo
df1 = dataframes[['item_id','app_name','genres']].copy()

In [13]:
df1['item_id'] = dataframes['item_id'].astype(int)

In [14]:
# Crea las variables dummi para la columna especificada
modelado = pd.get_dummies(df1, columns=['genres'], prefix="")
df_modelo = modelado.groupby(["item_id","app_name"]).sum().reset_index()
df_modelo.sample(3)

,item_id,app_name,_Action,_Adventure,_Animation &amp; Modeling,_Casual,_Design &amp; Illustration,_Education,_Free to Play,_Indie,_Pending classification,_RPG,_Racing,_Simulation,_Sports,_Strategy,_Utilities
678,319910,Trine 3: The Artifacts of Power,39,0,0,0,0,0,0,0,0,0,0,0,0,0,0
172,45400,Fortix,0,0,0,0,0,0,0,10,0,0,0,0,0,0,0
70,13250,Unreal Gold,10,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [15]:
df_modelo.to_parquet('D:/escritorio/STEAM/parquet/modelo.parquet')

In [16]:
# Calcula la similitud de coseno para todas las filas desde la tercer hasta la ultima columna
similitudes = cosine_similarity(df_modelo.iloc[:, 2:])

In [17]:
def recomendacion_juego(id:int):
    
    # Crea un df con los datos de el id pasado como parametro
    juego = df_modelo[df_modelo['item_id'] == id]
    
    # En caso que el ID no esté en el df, muestra el siguiente mensaje:
    if juego.empty:
        return f"El juego con ID {id} no se encuentra."

    # Obtiene el índice del juego en el DataFrame.
    idx = juego.index[0]

    # Tamaño de la muestra (asegúrate de que sea menor o igual al número de filas en df_modelo)
    muestra = min(2000, len(df_modelo))

    # Ajustamos la semilla aleatoria
    df_sample = df_modelo.sample(n=muestra, random_state=42)
    
    # Calculamos la similitud de contenido solo para el juego dado y la muestra
    sim_scores = cosine_similarity([df_modelo.iloc[idx, 2:]], df_sample.iloc[:, 2:])
    
    # Obtenemos las puntuaciones de similitud del juego dado con otros juegos
    sim_scores = sim_scores[0]

    # Ordenamos los juegos por similitud en orden descendente
    similar_games = [(i, sim_scores[i]) for i in range(len(sim_scores)) if i != idx]
    similar_games = sorted(similar_games, key=lambda x: x[1], reverse=True)

    # Obtenemos los 5 juegos más similares
    similar_game_indices = [i[0] for i in similar_games[:5]]

    # Listamos los juegos similares (solo nombres)
    similar_game_names = df_sample['app_name'].iloc[similar_game_indices].tolist()

    return {"Juegos similares": similar_game_names}


In [18]:
# Ejemplo de uso
recomendacion_juego(267610)

{'Juegos similares': ['X-Plane 10 Global - 64 Bit',
  'Farming Simulator 15',
  'Airport Simulator 2014',
  'Homeworld Remastered Collection',
  'Hacker Evolution']}